In [1]:
import numpy as np
import qokit
import scipy.optimize
from qokit import parameter_utils
from qokit.qaoa_objective_labs import get_qaoa_labs_objective

# Optimize QAOA parameters for LABS problem

We'll show an example of optimizing overlap with the ground state. Expectation value optimization is analogous.


First, let's get known parameters for largest depth $p$ that we have. QOKit has a built-in set of good LABS parameters.

In [2]:
N = 10
parameters = parameter_utils.get_best_known_parameters_for_LABS_wrt_overlap(N)
known_p = parameters.p.max()
print(f"Maximum p available for N={N} is {known_p}")
parameters.head()

Maximum p available for N=10 is 8


,N,p,overlap,gamma,beta,merit factor
35,10,1,0.127219,[0.0788139605],[-0.1899602111],1.923268
36,10,2,0.196232,"[0.06896497310000001, 0.1511922366]","[-0.1966433649, -0.1328573804]",2.174876
37,10,3,0.258441,"[0.063816785, 0.1396294913, 0.1538820941]","[-0.2082662354, -0.1383489014, -0.1215130558]",2.320238
38,10,4,0.318326,"[0.0655247286, 0.1319443875, 0.1430007569, 0.1...","[-0.20560510070000002, -0.1459282952, -0.12584...",2.445458
39,10,5,0.362467,"[0.0661013761, 0.1351407309, 0.136539053600000...","[-0.2261604879, -0.1482192504, -0.1413220406, ...",2.506400


Now that we have the parameters for `p_known`, let's find parameters for a larger depth! We will extrapolate the parameters that we already know to have a good starting point for optimization

In [3]:
p = known_p + 1
gamma, beta = parameter_utils.get_best_known_parameters_for_LABS_wrt_overlap_for_p(N, known_p)

The next cell will find an initial guess and create a function which maps parameters to the overlap. We can use `get_qaoa_labs_objective` with different parameterizations, but in this case, we'll use Fourier-transformed basis. 

The simulation will run on GPU if it is available, and if the problem is small, you will get a `NumbaPerformanceWarning`, which just indicates that you're not saturating capabilities of your GPU.

In [4]:
u, v = parameter_utils.to_fourier_basis(gamma, beta)
init_u, init_v = parameter_utils.extrapolate_parameters_in_fourier_basis(u, v, p, 1)
init_freq = np.hstack([init_u, init_v])
f = get_qaoa_labs_objective(N, p, parameterization='freq', objective='overlap')

In [5]:
print(f"Success probability at p={p} before optimization is {1-f(init_freq)}")

Success probability at p=9 before optimization is 0.49333386759027686


We will try optimizing further!

Specifically, we will extrapolate the parameters in the fourier basis using the interpolation scheme of https://arxiv.org/abs/1812.01041 and then optimize in the fourier basis

In [6]:
res = scipy.optimize.minimize(f, init_freq, method='COBYLA', options={'rhobeg': 0.01/N})

In [7]:
gamma, beta = parameter_utils.from_fourier_basis(res.x[:p], res.x[p:])
f_bg = get_qaoa_labs_objective(N, p, parameterization='gamma beta', objective='overlap')

print(f'Success probability at p={p} after optimization is {1-f_bg(gamma, beta)}')

Success probability at p=9 after optimization is 0.5243370462962005


Now we can use these parameters to solve LABS on a quantum computer!

---